In [1]:
import keras as keras
import tensorflow as tf
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.utils import pad_sequences
import pandas as pd
import numpy as np

In [2]:
csv_file_path = 'sinusoid_dataset.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file_path)
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split()])).values
labels = df['Label'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values
print(features.shape)
print(labels.shape)

(100000,)
(100000,)


In [3]:
padded_array = pad_sequences(labels, padding='post', maxlen=1)
print(padded_array.shape)
labels = np.vstack(padded_array)
features = np.vstack(features)
#scaler = MinMaxScaler()
#features = scaler.fit_transform(features)

(100000, 1)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_test_tensor = tf.convert_to_tensor(X_test, dtype=tf.float32)
y_test_tensor = tf.convert_to_tensor(y_test, dtype=tf.float32)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train_categorical shape:", y_train.shape)
print("y_test_categorical shape:", y_test.shape)

2023-12-10 11:25:19.466303: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 11:25:19.643065: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 11:25:19.643133: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 11:25:19.649100: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-10 11:25:19.649173: I external/local_xla/xla/stream_executor

X_train shape: (80000, 299)
X_test shape: (20000, 299)
y_train_categorical shape: (80000, 1)
y_test_categorical shape: (20000, 1)


In [8]:
model = keras.Sequential()
model.add(layers.LSTM(299, input_shape=(299, 1), return_sequences=True))
model.add(layers.BatchNormalization())  # Batch normalization can be helpful
model.add(layers.LSTM(128, return_sequences=True))
model.add(layers.BatchNormalization())
model.add(layers.LSTM(64, return_sequences=False))
model.add(layers.BatchNormalization())
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='tanh'))
model.add(layers.Dense(1))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 299, 299)          359996    
                                                                 
 batch_normalization_3 (Bat  (None, 299, 299)          1196      
 chNormalization)                                                
                                                                 
 lstm_4 (LSTM)               (None, 299, 128)          219136    
                                                                 
 batch_normalization_4 (Bat  (None, 299, 128)          512       
 chNormalization)                                                
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 batch_normalization_5 (Bat  (None, 64)               

In [9]:
model.compile(optimizer='adam',
              loss='mean_absolute_error',
              metrics=['mae', 'mse', 'mape'])
#overfitCallback = EarlyStopping(monitor='loss', min_delta=0, patience = 4)
#lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=1e-7)
model.fit(X_train_tensor, y_train_tensor, epochs=20, batch_size=256, validation_split=0.2)
test_loss, test_mae, test_mse, test_mape = model.evaluate(X_test_tensor, y_test_tensor)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test MAE: {test_mae:.4f}')
print(f'Test MSE: {test_mse:.4f}')
print(f'Test MAPE: {test_mape:.4f}')

Epoch 1/20
250/250 [==============================] - 43s 157ms/step - loss: 41.3292 - mae: 41.3292 - mse: 2456.4709 - mape: 75.6508 - val_loss: 35.3539 - val_mae: 35.3539 - val_mse: 1910.1240 - val_mape: 61.8682
Epoch 2/20
250/250 [==============================] - 38s 152ms/step - loss: 31.1475 - mae: 31.1475 - mse: 1611.6089 - mape: 50.8509 - val_loss: 28.5057 - val_mae: 28.5057 - val_mse: 1365.5098 - val_mape: 47.6703
Epoch 3/20
250/250 [==============================] - 37s 149ms/step - loss: 25.2104 - mae: 25.2104 - mse: 1155.6050 - mape: 40.8702 - val_loss: 22.5856 - val_mae: 22.5856 - val_mse: 979.1346 - val_mape: 35.0796
Epoch 4/20
250/250 [==============================] - 39s 158ms/step - loss: 20.2251 - mae: 20.2251 - mse: 826.9146 - mape: 30.5223 - val_loss: 19.0963 - val_mae: 19.0963 - val_mse: 702.6180 - val_mape: 30.0776
Epoch 5/20
250/250 [==============================] - 37s 149ms/step - loss: 16.2255 - mae: 16.2255 - mse: 585.2357 - mape: 23.9884 - val_loss: 14.3978

In [10]:
csv_file_path = 'samplesreal.csv'  # Replace with the actual path to your CSV file
df = pd.read_csv(csv_file_path)

# Extract features and labels
features = df['Feature'].apply(lambda x: np.array([float(val.strip("[]")) for val in x.split(',')])).values

# Convert the string of up to 5 numbers to a NumPy array of floats
labels = df['Label'].values
labels = np.vstack(labels)
features = np.vstack(features)

print(labels.shape)
print(features.shape)

means = np.mean(features, axis=1, keepdims=True)
print(means)
features = features - means
#change to 50
features = features * 50
print("Real Sinusoid Tests:\n")
for i in range(8):
  print(labels[i:i+1])
  print(model.predict(features[i:i+1]))
print("Synthetic Sinusoid Tests:\n")
for i in range(100):
  print(y_test[i:i+1])
  print(model.predict(X_test[i:i+1]))

(8, 1)
(8, 299)
[[2.39770275]
 [2.37782306]
 [2.37742957]
 [2.37666683]
 [2.37638921]
 [2.37874752]
 [2.37809528]
 [2.37756972]]
Real Sinusoid Tests:

[[1]]
1/1 [==============================] - 1s 725ms/step
[[1.594873]]
[[10]]
1/1 [==============================] - 0s 73ms/step
[[9.389566]]
[[19]]
1/1 [==============================] - 0s 33ms/step
[[18.276806]]
[[27]]
1/1 [==============================] - 0s 41ms/step
[[26.469257]]
[[36]]
1/1 [==============================] - 0s 65ms/step
[[35.501297]]
[[45]]
1/1 [==============================] - 0s 34ms/step
[[45.40043]]
[[54]]
1/1 [==============================] - 0s 36ms/step
[[53.19606]]
[[90]]
1/1 [==============================] - 0s 34ms/step
[[91.0962]]
Synthetic Sinusoid Tests:

[[51]]
1/1 [==============================] - 0s 41ms/step
[[50.510677]]
[[54]]
1/1 [==============================] - 0s 49ms/step
[[53.655098]]
[[7]]
1/1 [==============================] - 0s 49ms/step
[[7.565708]]
[[16]]
1/1 [===============